In [1]:
import itertools
import lime_explainer as lime

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
names = [
  'price',
  'date_utc',
  'day',
  'hour',
  'prev_week_min',
  'prev_week_25th',
  'prev_week_50th',
  'prev_week_75th',
  'prev_week_max'
]
w_names = [
  'temperature',
  'wind_speed_100m',
  'wind_direction_100m',
  'air_density',
  'precipitation',
  'wind_gust',
  'radiation',
  'wind_speed',
  'wind_direction',
  'pressure'
]

w_names = [ ['loc'+str(i)+"_"+n for n in w_names] for i in range(18)]

w_names = list(itertools.chain.from_iterable(w_names))
feature_names = names + w_names

In [3]:
explainer = lime.Explainer(model_type='regression')

In [4]:
explainer.load_data(
  gcs_path = 'gs://energyforecast/data/csv/MLDataTrain.csv',
  target_idx = 0,
  features_to_use = range(2,189),
  categorical_features = [2,3],
  feature_names = feature_names,
  skip_first=True
)

In [5]:
explainer.create_cmle_client(
  gcp_project = 'energy-forecasting',
  gcp_model = 'energyforecaster',
  gcp_model_version = 'new_energy',
  padding = (2,0)
)

In [6]:
record, df = explainer.explain_random_record(numeric_rows=[])

Intercept 0.5055876368704718
Prediction_local [0.41062113]
Right: 0.3514913022518158


In [7]:
print(record)

[5.0, 1.0, 0.2687, 0.371, 0.4843, 0.5788000000000001, 0.6709999999999999, 10.0, 7.8, 198.2, 1.17, 0.0, 12.2, 0.0, 3.0, 189.4, 948.7, 10.7, 5.3, 220.1, 1.14, 0.0, 9.3, 0.0, 2.6, 214.1, 929.3, 14.6, 7.8, 232.2, 1.21, 0.0, 9.5, 0.0, 4.2, 228.8, 1001.0, 16.6, 4.6, 263.7, 1.23, 0.0, 5.2, 0.0, 3.5, 262.7, 1018.9, 4.7, 5.0, 217.9, 1.06, 0.0, 3.7, 0.0, 2.8, 213.6, 846.6, 8.7, 8.5, 249.4, 1.14, 0.0, 9.6, 0.0, 4.6, 242.3, 926.0, 13.0, 6.8, 254.5, 1.21, 0.0, 7.5, 0.0, 3.3, 249.8, 990.9, 13.8, 5.2, 218.8, 1.24, 0.0, 4.4, 0.0, 2.6, 202.7, 1017.6, 8.0, 7.8, 213.5, 1.12, 0.0, 4.9, 0.0, 4.1, 206.8, 906.3, 4.5, 3.3, 279.1, 1.1, 0.0, 2.5, 0.0, 1.8, 280.6, 878.3, 11.5, 8.9, 251.1, 1.16, 0.0, 8.8, 0.0, 4.5, 243.6, 944.9, 9.2, 2.5, 257.0, 1.1, 0.0, 2.3, 0.0, 1.6, 259.7, 890.5, 10.5, 7.2, 170.1, 1.15, 0.0, 5.8, 0.0, 4.5, 177.1, 938.9, 12.1, 12.6, 211.3, 1.08, 0.0, 15.5, 0.0, 7.1, 208.4, 880.3, 15.1, 8.9, 233.6, 1.14, 0.0, 9.9, 0.0, 4.3, 223.4, 939.2, 17.4, 2.1, 245.7, 1.19, 0.0, 1.6, 0.0, 1.4, 234.5, 992.2,

In [8]:
df

,representation,weight
0,0.37 < prev_week_25th <= 0.41,-0.059130
1,loc4_wind_speed_100m > 4.40,-0.032551
2,1.04 < loc4_air_density <= 1.06,-0.022685
3,11.50 < loc2_temperature <= 16.90,0.019015
4,238.30 < loc10_wind_direction <= 262.40,0.000384


In [9]:
mod_record = ','.join(['5','12'] + [str(e) for e in record[2:]])
print('Mod Record: {}\n\n'.format(mod_record))
_, mod_df = explainer.explain_record(mod_record)

Mod Record: 5,12,0.2687,0.371,0.4843,0.5788000000000001,0.6709999999999999,10.0,7.8,198.2,1.17,0.0,12.2,0.0,3.0,189.4,948.7,10.7,5.3,220.1,1.14,0.0,9.3,0.0,2.6,214.1,929.3,14.6,7.8,232.2,1.21,0.0,9.5,0.0,4.2,228.8,1001.0,16.6,4.6,263.7,1.23,0.0,5.2,0.0,3.5,262.7,1018.9,4.7,5.0,217.9,1.06,0.0,3.7,0.0,2.8,213.6,846.6,8.7,8.5,249.4,1.14,0.0,9.6,0.0,4.6,242.3,926.0,13.0,6.8,254.5,1.21,0.0,7.5,0.0,3.3,249.8,990.9,13.8,5.2,218.8,1.24,0.0,4.4,0.0,2.6,202.7,1017.6,8.0,7.8,213.5,1.12,0.0,4.9,0.0,4.1,206.8,906.3,4.5,3.3,279.1,1.1,0.0,2.5,0.0,1.8,280.6,878.3,11.5,8.9,251.1,1.16,0.0,8.8,0.0,4.5,243.6,944.9,9.2,2.5,257.0,1.1,0.0,2.3,0.0,1.6,259.7,890.5,10.5,7.2,170.1,1.15,0.0,5.8,0.0,4.5,177.1,938.9,12.1,12.6,211.3,1.08,0.0,15.5,0.0,7.1,208.4,880.3,15.1,8.9,233.6,1.14,0.0,9.9,0.0,4.3,223.4,939.2,17.4,2.1,245.7,1.19,0.0,1.6,0.0,1.4,234.5,992.2,11.6,9.2,174.8,1.07,0.0,9.7,0.0,4.9,174.2,877.5,11.6,9.2,174.8,1.07,0.0,9.7,0.0,4.9,174.2,877.5


Intercept 0.5238604343961857
Prediction_local [0.3916833]
Ri

In [10]:
mod_df

,representation,weight
0,90.47 < loc1_wind_direction_100m <= 239.30,-0.054155
1,6.60 < loc0_temperature <= 11.00,-0.044444
2,88.38 < loc1_wind_direction <= 234.50,-0.029648
3,1.20 < loc2_air_density <= 1.23,-0.023052
4,loc0_radiation <= 0.00,0.019121
